In [24]:
# import modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use(['science','no-latex'])
import pandas_datareader as web
import datetime as dt
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import talib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,balanced_accuracy_score,f1_score,average_precision_score,precision_score,confusion_matrix, r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from keras.models import Sequential
import time
import types
from keras.layers import Dense
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDRegressor, Lasso, ElasticNet, Perceptron, Ridge
from sklearn import svm
from tensorflow.keras import layers
from sklearn.feature_selection import RFE, SelectFromModel, SelectPercentile, mutual_info_regression, mutual_info_classif, SelectFpr, VarianceThreshold, RFECV, SelectKBest, f_classif, f_regression, chi2
#from yellowbrick.model_selection.importances import FeatureImportances
from sklearn.decomposition import PCA
from xgboost import XGBRegressor


In [25]:
#Getting Data

start="1-1-1993"
end=dt.datetime.today()

#ticker = "^GSPC"
#ticker = "^BSESN"
#ticker = "UBSG.SW"
ticker = "^IXIC"
#ticker = "^N225"
#ticker = "EZA"

data = web.DataReader(ticker, "yahoo", start, end)
data = data[ : ]###

#Adding Pct Change/ Mov
data['Pct Change'] = data['Close'].pct_change()*100
data['Movement']=None
m = 0.2
for i in range(0, len(data['Pct Change'])):
    if data['Pct Change'].iloc[i]<-m:
        data['Movement'].iloc[i]=-1
    elif data['Pct Change'].iloc[i]>m:
        data['Movement'].iloc[i]=1
    else:
        data['Movement'].iloc[i]=0

/Users/advikchaudhary/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [26]:
#Feature Engineering

#Basic Features
fdata = pd.DataFrame(index = data.index)
fdata['DReturn'] = (data['Close']/data['Open']).apply(np.log)
fdata['Vol'] = talib.OBV(data['Close'], data['Volume'])
fdata['High'] = data['High'].apply(np.log)
fdata['Pct Change'] = data['Pct Change']

period = np.arange(1,31)

#Avg
for i in range(1,200):
    fdata[f'MA {i}'] = talib.MA(data['Close'], timeperiod = i, matype = 0)    
for i in range(2, len(period)):
    fdata[f'SMA {i}']  =  talib.SMA(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'DEMA {i}']  =  talib.DEMA(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'TEMA {i}']  =  talib.TEMA(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'EMA {i}']  =  talib.EMA(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'STDDEV {i}']  =  talib.STDDEV(data['Close'], timeperiod = i, nbdev = 1)
for i in range(2, len(period)):
    fdata[f'SAREXT {i}']  =  talib.SAREXT(data['High'], data['Low'], startvalue = i, offsetonreverse = 0, 
                                        accelerationinitlong = 0, accelerationlong = 0, accelerationmaxlong = 0, 
                                        accelerationinitshort = 0, accelerationshort = 0, accelerationmaxshort = 0)
for i in range(2, len(period)):
    fdata[f'WMA {i}']  =  talib.WMA(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata['Price Volitility'] = data['Pct Change'].rolling(i).mean()
    
#Price
fdata['AVGPRICE']  =  talib.AVGPRICE(data['Open'],data['High'], data['Low'], data['Close']) 
fdata['MEDPRICE']  =  talib.MEDPRICE(data['High'], data['Low'])
fdata['TYPPRICE']  =  talib.TYPPRICE(data['High'], data['Low'], data['Close'])  
fdata['WCLPRICE']  =  talib.TYPPRICE(data['High'], data['Low'], data['Close'])  

#Moementum
for i in range(2, len(period)):
    fdata[f'ADX {i}']  =  talib.ADX(data['High'],data['Low'], data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'ADXR {i}']  =  talib.ADXR(data['High'],data['Low'], data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'APO {i}']  =  talib.APO(data['Close'], fastperiod = i, slowperiod = i*2, matype = 0)
for i in range(2, len(period)):
    fdata[f'AROON D{i}'], fdata[f'AROON U{i}']  =  talib.AROON(data['High'],data['Low'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'AROONOSC {i}']  =  talib.AROONOSC(data['High'],data['Low'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'BOP {i}']  =  talib.BOP(data['Open'], data['High'],data['Low'], data['Close'])
for i in range(2, len(period)):
    fdata[f'CCI {i}']  =  talib.CCI(data['High'],data['Low'], data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'CMO {i}']  =  talib.CMO(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'DX {i}']  =  talib.DX(data['High'],data['Low'], data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'MACD MACD {i}'], fdata[f'MACD SIGNAL{i}'], fdata[f'MACD HIST{i}']  =  talib.MACD(data['Close'], fastperiod = i, slowperiod = i*2, signalperiod = i)
for i in range(2, len(period)):
    fdata[f'MACDEXT MACD {i}'], fdata[f'MACDEXT SIGNAL{i}'], fdata[f'MACDEXT HIST{i}']  =  talib.MACDEXT(data['Close'], fastperiod = i, fastmatype = 0, slowperiod = i*2, 
                                                                                                       slowmatype = 0, signalperiod = 9, signalmatype = 0)
for i in range(2, len(period)):
    fdata[f'MACDFIX MACD {i}'], fdata[f'MACDFIX SIGNAL{i}'], fdata[f'MACDFIX HIST{i}']  =  talib.MACDFIX(data['Close'], signalperiod = 9)
for i in range(2, len(period)):
    fdata[f'MFI {i}']  =  talib.MFI(data['High'],data['Low'], data['Close'], data['Volume'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'MINUS_DI {i}']  =  talib.MINUS_DI(data['High'],data['Low'], data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'MINUS_DM {i}']  =  talib.MINUS_DM(data['High'],data['Low'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'MOM {i}']  =  talib.MOM(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'PLUS_DI {i}']  =  talib.PLUS_DI(data['High'],data['Low'], data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'PLUS_DM {i}']  =  talib.PLUS_DM(data['High'],data['Low'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'PPO {i}']  =  talib.PPO(data['Close'], fastperiod = i, slowperiod = i*2, matype = 0)
for i in range(2, len(period)):
    fdata[f'ROC {i}']  =  talib.ROC(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'ROCP {i}']  =  talib.ROCP(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'ROCR {i}']  =  talib.ROCR(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'ROCR100 {i}']  =  talib.ROCR100(data['Close'], timeperiod = i)
for i in range(8, len(period), 6):
    fdata[f'RSI {i}']  =  talib.RSI(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'STOCH SLOWK {i}'], fdata[f'STOCH SLOWD {i}']  =  talib.STOCH(data['High'],data['Low'], data['Close'], 
                                                                       fastk_period = 5, slowk_period = 3, slowk_matype = 0, slowd_period = 3, slowd_matype = 0)
for i in range(2, len(period)):
    fdata[f'STOCHF FASTK {i}'], fdata[f'STOCHF FASTD {i}']  =  talib.STOCHF(data['High'],data['Low'], data['Close'], 
                                                                          fastk_period = i, fastd_period = round(i/2), fastd_matype = 0)
for i in range(2, len(period)):
    fdata[f'STOCHRSI FASTK {i}'], fdata[f'STOCHRSI FASTD {i}']  =  talib.STOCHRSI(data['Close'], timeperiod = i*3, 
                                                                                fastk_period = round(i/2), fastd_period = round(i/3), fastd_matype = 0)
for i in range(2, len(period)):
    real  =  talib.TRIX(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'ULTOSC {i}']  =  talib.ULTOSC(data['High'],data['Low'], data['Close'], timeperiod1 = i, timeperiod2 = i*2, timeperiod3 = i*3)
for i in range(2, len(period)):
    fdata[f'WILLR {i}']  =  talib.WILLR(data['High'],data['Low'], data['Close'], timeperiod = i)

#Volitility
for i in range(2, len(period)):
    fdata[f'ATR {i}']  =  talib.ATR(data['High'],data['Low'], data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'NATR {i}']  =  talib.NATR(data['High'],data['Low'], data['Close'], timeperiod = i)  
fdata['TRANGE']  =  talib.TRANGE(data['High'],data['Low'], data['Close'])

#Cycle Indicators
fdata['HT_DCPERIOD']  =  talib.HT_DCPERIOD(data['Close'])
fdata['HT_DCPHASE']  =  talib.HT_DCPHASE(data['Close'])
fdata['HT_TRENDMODE']  =  talib.HT_TRENDMODE(data['Close'])

#Statistic Functions
for i in range(2, len(period)):
    fdata[f'BETA {i}']  =  talib.BETA(data['High'],data['Low'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'CORREL {i}']  =  talib.CORREL(data['High'],data['Low'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'LINEARREG {i}']  =  talib.LINEARREG(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'LINEARREG_SLOPE {i}']  =  talib.LINEARREG_SLOPE(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'STDDEV {i}']  =  talib.STDDEV(data['Close'], timeperiod = i, nbdev = 1)
for i in range(2, len(period)):
    fdata[f'TSF {i}']  =  talib.TSF(data['Close'], timeperiod = i)
for i in range(2, len(period)):
    fdata[f'VAR {i}']  =  talib.VAR(data['Close'], timeperiod = i, nbdev = 1)

#Volume Indicator Functions
for i in range(2, len(period)):
    fdata[f'ADOSC {i}']  =  talib.ADOSC(data['High'], data['Low'], data['Close'], data['Volume'], fastperiod = i, slowperiod = i*2)
fdata['AD'] = talib.AD(data['High'], data['Low'], data['Close'], data['Volume'])
fdata['OBV'] = talib.OBV(data['Close'], data['Volume'])

fdata['SINE'] = np.sin(np.arange(len(fdata)))
fdata = fdata.dropna()
len(fdata.columns)

1731

In [27]:
target = 'Pct Change'

In [164]:
#Feature Selection with scikit regr SPY
selector  =  SelectKBest(f_regression,  k = 5)
sf  =  selector.fit_transform(fdata[:-1],  fdata[target].shift(-1).dropna())
fnum  =  selector.get_support(indices = True)

/Users/advikchaudhary/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/Users/advikchaudhary/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/advikchaudhary/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/Users/advikchaudhary/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/advikchaudhary/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/a

In [ ]:
#Data Splitting

X = fdata.iloc[:, fnum[:]].astype('float')
tmrw = X[-1:]
X = X[:-1]
y = fdata[target].shift(-1).dropna().astype('float')
splitp = int(len(X)*0.75)

X_train =  X[:splitp]
X_test  =  X[splitp:]
y_train =  y[:splitp]
y_test =  y[splitp:]

print(len(X_test.columns))

model = RandomForestRegressor(random_state = 0)
model.fit(X_train,  y_train)
pred = model.predict(X_test)
tpred = model.predict(tmrw)
pred = pd.DataFrame(data = pred,  index = X_test.index,  columns = ["Prediction"])
tpred = pd.DataFrame(data = tpred,  index = tmrw.index)

X_test.columns